In [18]:
import numpy as np
import pandas as pd

In [19]:
from TWMR import TWMR
from revTWMR import revTWMR

# TWMR package usage

We shall provide matrix of standardized effects of QTLS on gene expression.

In [20]:
effect = pd.read_csv('data/TWMR/ENSG00000000419.matrix', sep='\t', index_col=0)

Matrix looks like this.

Each column represents one gene, last column called `GWAS` represents standardized effect of each QTL on studied trait.  
Each row is QTL with qtl labels as indicies for rows.

In [21]:
effect.head()

,ENSG00000000419,ENSG00000101126,GWAS
SNPS,,,
rs7268202,0.000000,-0.028909,0.002336
rs6013040,-0.026739,0.032592,-0.005177
rs2426214,-0.051089,0.000000,0.004599


Also we have to provide LD correlation matrix, as numpy array:

In [22]:
ld = pd.read_csv('data/TWMR/ENSG00000000419.ld', sep='\t', header=None).to_numpy().astype(np.float32)

which looks like this:

In [23]:
ld

array([[1.       , 0.0487694, 0.151668 ],
       [0.0487694, 1.       , 0.118453 ],
       [0.151668 , 0.118453 , 1.       ]], dtype=float32)

And set a sizes of GWAS and QTLs studies used for analysis:

In [24]:
nGWAS=239087
nQTLs = 32000

Everything is in place, run analysis:

In [25]:
result = TWMR(
        beta=effect.drop('GWAS', axis=1).to_numpy(), 
        gamma=effect['GWAS'].to_numpy(), 
        nEQTLs=nQTLs, 
        NGwas=nGWAS, 
        ldMatrix=ld, 
        pseudoInverse=False, 
        device='cpu')

Result is a standard python's `namedtuple`  
Attributes:
* Alpha : Causal effect estimated by TWMR.
* Se : Standard error of Alpha.
* Pval : P-value calculated from Alpha and Se.
* D : Cohrian Q statistics for QTLs.
* HetP : P-value for heterogeneity test.   

And, nicer output:

In [26]:
result_df = pd.DataFrame()
result_df['Gene'] = effect.columns.drop('GWAS')
result_df['Alpha'] = result.Alpha
result_df['Standard error'] = result.Se
result_df['p-value'] = result.Pval
result_df['Heterogenity p-value'] = result.HetP
display(result_df)

,Gene,Alpha,Standard error,p-value,Heterogenity p-value
0,ENSG00000000419,-0.005793,0.006162,0.347168,1.0
1,ENSG00000101126,-0.013730,0.005420,0.011305,1.0


------------------------

# RevTWMR package usage


Again, we shall provide matrix of standardized effects of QTLS on gene expression.

In [27]:
effect = pd.read_csv("data/revTWMR/effect.matrix.tsv", sep='\t', index_col=0)


This time, table contains standardized QTL effect for each gene, standardized effect for GWAS, standard error of the effect and number of samples in GWAS study.

In [28]:
effect.head()

,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000270167,ENSG00000270170,ENSG00000270172,ENSG00000270175,ENSG00000270177,ENSG00000270179,ENSG00000270184,BETA_GWAS,SE,N
SNP,,,,,,,,,,,,,,,,,,,,,
rs1421085,0.010424,-0.000291,-0.009326,-0.006210,0.008370,0.003784,-0.002304,0.002775,0.001868,0.011122,...,-0.009427,0.000016,-0.013146,0.004121,0.009272,0.000837,0.000911,0.050996,0.001667,359983
rs12967135,-0.007225,0.002163,0.003903,0.003141,-0.001353,0.000684,-0.007229,-0.006119,-0.001645,0.002090,...,-0.012976,-0.000828,0.015115,-0.000321,-0.004691,0.016114,0.010373,0.032377,0.001667,359983
rs12463617,-0.008345,-0.008935,0.004012,0.003909,-0.003861,0.000139,0.006580,0.008313,-0.001529,-0.001600,...,-0.020190,-0.000314,0.014615,0.001792,-0.018654,0.006185,0.019847,-0.028158,0.001667,359983
rs543874,0.001516,-0.004459,-0.011000,0.000491,-0.000259,-0.003478,0.002421,-0.005493,-0.008331,-0.008748,...,-0.030773,-0.014313,-0.011587,0.023924,0.002606,-0.005619,-0.040796,0.027852,0.001667,359983
rs713586,-0.005947,-0.004387,-0.006228,0.005699,-0.006790,-0.010466,0.000394,-0.005812,-0.005693,-0.004187,...,-0.009529,-0.019617,0.002262,-0.030201,0.011784,0.000516,0.026635,0.024829,0.001667,359983


In [29]:
gwasEffect = effect.BETA_GWAS.values
nGwas = effect.N.values

This time we have to provide `standardized` study size for each gene.

In [30]:
sample_size = pd.read_csv("data/revTWMR/genes.N.tsv", sep='\t', index_col=0).N.to_dict()

Let's run it for first 5 genes:

In [31]:
results = {}
for gene in effect.drop(['BETA_GWAS', 'SE', 'N'], axis=1).columns[:5]:    
    effectTbl = effect.drop(['BETA_GWAS', 'SE', 'N'], axis=1)[gene].values
    nQtls = sample_size[gene]
    
    result = revTWMR(
        effectTbl, 
        gwasEffect, 
        qtlLabels=effect.index.values, 
        gwasSizes=nGwas, 
        qtlExpSize=nQtls,
        pValIterativeThreshold=0.05, 
        pseudoInverse=False, device='cpu')
    
    results[gene] = {
        'Alpha Original': result.Alpha,
        'SE Original': result.Se,
        'P Value Original': result.Pval,
        'N Original': result.N,
        'P heterogeneity Original': result.HetP,
        'Alpha': result.AlphaIterative,
        'SE': result.SeIterative,
        'P': result.PvalIterative,
        'P heterogeneity': result.HetPIterative,
        'N': len(result.rsname)
    }

This time result would be:

In [32]:
pd.DataFrame.from_dict(results, orient='index')

,Alpha Original,SE Original,P Value Original,N Original,P heterogeneity Original,Alpha,SE,P,P heterogeneity,N
ENSG00000000003,-0.024712,0.068517,0.718346,101.0,1.000000,-0.024712,0.068517,0.718346,1.000000,101
ENSG00000000419,-0.042308,0.035751,0.236646,123.0,0.957932,-0.042308,0.035751,0.236646,0.957932,123
ENSG00000000457,0.007607,0.035183,0.828817,123.0,0.942955,0.007607,0.035183,0.828817,0.942955,123
ENSG00000000460,-0.042934,0.035112,0.221417,123.0,0.999823,-0.042934,0.035112,0.221417,0.999823,123
ENSG00000000938,0.023117,0.035309,0.512657,122.0,0.945195,0.023117,0.035309,0.512657,0.945195,122
